In [21]:
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
from tqdm import tqdm

In [22]:
DRIVER_SITE = '/Applications/chromedriver'
ORIGINAL_URL = 'https://keirin.netkeiba.com/db/search_result/race.html?word=&start_year=none&start_mon=none&end_year=none&end_mon=none&jyo=&sort=1&submit='
CURRENT_DIR = '/Users/yamazakiyuuta/Library/Mobile Documents/com~apple~CloudDocs/Product/keirin-pred'
PARTICIPANTS_FILENAME = 'participants.csv'
PARTICIPANTS_HEADER = (
    '日付, 開催場所, ラウンド, グレード, レースグループ, レース名, 発走時間, 距離, 周回,'
    '選手名, 着順, 枠番, 車番, 着差, 上り, 決, SB\n'
)
RACE_RESULTS_FILENAME = 'race_results.csv'
RACE_RESULTS_HEADER = (
    '日付, 開催場所, ラウンド, グレード, レースグループ, レース名, 発走時間, 距離, 周回,'
    '2車複, 2車単, ワイド, 3連複, 3連単\n'
)

In [23]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome('/Applications/chromedriver', options=options)
# driver = webdriver.Chrome(DRIVER_SITE)
driver.get(ORIGINAL_URL)
sleep(1)

In [24]:
def get_date_and_venue(soup):
    
    date_and_venue = []

    # 開催日と開催場所の取得
    for data in soup.find_all(class_ = 'DataBox_01'):
        (date, venue) = data.p.get_text(strip=True).split()
        date_and_venue.append((date, venue))

    return date_and_venue

In [25]:
def date_split(date_str, date_term):

    dates = []
    
    date_split = date_str.split('～')
    dates.append(date_split[0])
    tdate = datetime.strptime(date_split[0], '%Y/%m/%d')

    for _ in range(1, date_term):
        tdate += timedelta(days=1)
        dates.append(datetime.strftime(tdate, '%Y/%m/%d'))

    return dates


In [26]:
def get_race_basic_data(soup):

    race_base_info = []

    # ラウンド、グレード、レースグループ、レース名、発走時間、距離、周回の取得
    race_base_info.append(soup.find(class_ = 'Race_Num').get_text(strip=True))
    race_base_info.append(soup.find(class_ = 'Icon_GradeType').get_text(strip=True))
    race_base_info.append(soup.find(class_ = 'Race_GroupName').get_text(strip=True))
    race_base_info.append(soup.find(class_ = 'Race_Name').get_text(strip=True))
    for i, data in enumerate(soup.find(class_ = 'Race_Data').get_text(strip=True).split()):
        if i == 1 or i == 4 or i == 5:
            race_base_info.append(data)

    return race_base_info

In [27]:
def get_participants(soup):
    
    participants = []

    # 選手名、着差、上り、決、SBの取得
    for player_info in soup.find_all('tr', attrs={'class', 'PlayerList'}):
        participant = []
        participant.append(player_info.find(class_ = 'Player01').get_text(strip=True))
        for race_info in player_info.find_all('td', attrs={'class', 'RaceCardCell01'}):
            participant.append(race_info.get_text(strip=True))
        participants.append(participant)

    return participants

In [28]:
def get_race_result(soup):
    
    race_result = []

    # 2車複の取得
    result_str = ''
    for result in soup.find_all(class_ = 'Umaren'):
        result_str += result + ';'
    race_result.append(result_str.rstrip(';'))

    # 2車単の取得
    result_str = ''
    for result in soup.find_all(class_ = 'Umatan'):
        result_str += result + ';'
    race_result.append(result_str.rstrip(';'))

    # ワイドの取得
    result_str = ''
    for result in soup.find_all(class_ = 'Wide'):
        result_str += result + ';'
    race_result.append(result_str.rstrip(';'))

    # 3連複の取得
    result_str = ''
    for result in soup.find_all(class_ = 'Fuku3'):
        result_str += result + ';'
    race_result.append(result_str.rstrip(';'))

    # 3連単の取得
    result_str = ''
    for result in soup.find_all(class_ = 'Tan3'):
        result_str += result + ';'
    race_result.append(result_str.rstrip(';'))

    return race_result

In [29]:
def date_and_race_page_scraping(driver, date_and_venue, participants_file, race_results_file):

    # レース一覧ページのURLの取得
    date_and_race_url = driver.current_url

    # 開催日リンクの取得
    date_list = driver.find_elements_by_xpath('//div[@class="Tab_RaceDaySelect p00"]/ul/li/a')

    # 日付の分割
    dates = date_split(date_and_venue[0], len(date_list))

    # 開催日単位での処理
    for date_count in range(len(date_list)):

        # 開催日リンクの取得(古いセッション参照対応用)
        trans_date_list = driver.find_elements_by_xpath('//div[@class="Tab_RaceDaySelect p00"]/ul/li/a')

        # 開催日ページへの遷移
        trans_date_list[date_count].click()
        sleep(1)

        # 開催日単位レース詳細ページ要素の取得(古いセッション参照対応用)
        date_and_race_list = driver.find_elements_by_xpath('//div[@class="RaceList_SlideBoxItem"]')

        # 開催日単位レース詳細ページリンクの取得
        race_list = date_and_race_list[date_count].find_elements_by_tag_name('a')

        # レース詳細ページ単位での処理
        for race_num in range(len(race_list)):
            # 開催日単位レース詳細ページ要素の取得(古いセッション参照対応用)
            trans_date_and_race_list = driver.find_elements_by_xpath('//div[@class="RaceList_SlideBoxItem"]')

            # 開催日単位レース詳細ページリンクの取得(古いセッション参照対応用)
            trans_race_list = trans_date_and_race_list[date_count].find_elements_by_tag_name('a')

            # レース詳細ページへの遷移
            trans_race_list[race_num].click()
            sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # レース基本データの取得
            race_basic_data = get_race_basic_data(soup)

            # 出場選手データの取得
            participants = get_participants(soup)

            # レース結果データの取得
            race_result = get_race_result(soup)

            for participant in participants:
                participants_file.write(
                    dates[date_count] + 
                    ',' + date_and_venue[1] + 
                    ',' + ','.join(race_basic_data) + 
                    ',' + ','.join(participant) + '\n'
                )

            race_results_file.write(
                dates[date_count] + 
                ',' + date_and_venue[1] + 
                ',' + ','.join(race_basic_data) + 
                ',' + ','.join(race_result) + '\n'
            )

            # レース一覧ページへ戻る
            driver.get(date_and_race_url)
            sleep(1)
            # 開催日ページへの遷移
            trans_date_list = driver.find_elements_by_xpath('//div[@class="Tab_RaceDaySelect p00"]/ul/li/a')
            trans_date_list[date_count].click()
            sleep(1)

In [30]:
try:
    participants_file = open(os.path.join(CURRENT_DIR, PARTICIPANTS_FILENAME), 'w')
    participants_file.write(PARTICIPANTS_HEADER)

    race_results_file = open(os.path.join(CURRENT_DIR, RACE_RESULTS_FILENAME), 'w')
    race_results_file.write(RACE_RESULTS_HEADER)

    # 全ページ数の取得
    driver.find_element_by_link_text('最後').click()
    sleep(1)
    total_page_num = int(driver.find_element_by_xpath('//a[@class="Page_Active"]').text)
    driver.get(ORIGINAL_URL)
    sleep(1)

    race_group_url = ORIGINAL_URL

    for _ in tqdm(range(total_page_num)):
        # 開催日と開催場所の取得
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        date_and_venue = get_date_and_venue(soup)
        
        # レースグループリンクの取得
        race_group_list = driver.find_elements_by_xpath('//ul[@class="CommonList_01"]/li/div/a')

        # レースグループ単位での処理
        for race_group_count in range(len(race_group_list)):
            # レース一覧ページへの遷移
            trans_race_group_list = driver.find_elements_by_xpath('//ul[@class="CommonList_01"]/li/div/a')
            trans_race_group_list[race_group_count].click()
            sleep(1)

            # レース一覧ページのWebスクレイピング
            date_and_race_page_scraping(driver,
                                        date_and_venue[race_group_count],
                                        participants_file,
                                        race_results_file)

            # レースグループ一覧ページへ戻る
            driver.get(race_group_url)
            sleep(1)

        # 次のレースグループ一覧ページへ遷移する
        driver.find_element_by_link_text('次へ').click()
        sleep(1)
        race_group_url = driver.current_url

except Exception as e:
    print('エラー発生!')
    print(driver.current_url)
    print(e)
finally:
    participants_file.close()
    race_results_file.close()

  0%|          | 1/689 [23:37<270:51:05, 1417.25s/it]


KeyboardInterrupt: 

In [ ]:
driver.quit()